In [1]:
import pandas as pd
from collections import defaultdict
import warnings
import visualize
import neat
import multiprocessing
import os
import matplotlib.pyplot as plt

import numpy as np

In [2]:
necoc = 26
generations = int(3000/necoc)
generations

115

In [3]:
df = pd.read_csv('letter-recognition.data',header=None)

train_df = df[:1600]
test_df = df[16000:16000+400]

X = train_df.ix[:,1:16].to_numpy() 
y = np.array([ord(l)-ord('A') for l in train_df.ix[:,0].to_numpy()])

testX = test_df.ix[:,1:16].to_numpy() 
testy = np.array([ord(l)-ord('A') for l in test_df.ix[:,0].to_numpy()])

In [4]:
def softmax(x, axis=1):
    # 计算每行的最大值
    row_max = x.max(axis=axis)
 
    # 每行元素都需要减去对应的最大值，否则求exp(x)会溢出，导致inf情况
    row_max=row_max.reshape(-1, 1)
    x = x - row_max
 
    # 计算e的指数次幂
    x_exp = np.exp(x)
    x_sum = np.sum(x_exp, axis=axis, keepdims=True)
    s = x_exp / x_sum
    return s

In [5]:
def get_winner(class1):
    
    numbers = [*range(26)] # number 2 > number 1
    number_of_classification = len(numbers)
    #number_of_sampling = 100

    list_y = y.tolist()
    digits_indexes = []
    for digit in numbers:
        li = [i for i in range(len(list_y)) if list_y[i] == digit]
        digits_indexes.extend([li])

        
    class2 = []

    class1_indexes = []
    class2_indexes = []

    for i in numbers:
        if i in class1:
            class1_indexes.extend(digits_indexes[i])
        else:
            class2_indexes.extend(digits_indexes[i])
            class2.append(i)

    class1_label = [1] * len(class1_indexes)
    class2_label = [0] * len(class2_indexes)

    print("number of class1:{}".format(len(class1_label)))
    print("number of class2:{}".format(len(class2_label)))

    samplesize = 1600
    x_inputs =[tuple(c) for c in X[class1_indexes].tolist()] + [tuple(c) for c in X[class2_indexes].tolist()]
    x_outputs = [tuple([c]) for c in class1_label + class2_label]
    
    
    def eval_genomes(genomes, config):
        for genome_id, genome in genomes:
            net = neat.nn.FeedForwardNetwork.create(genome, config)

            outputs = []
            for xi in x_inputs:
                output = net.activate(xi)
                outputs.append(output)


            px_outputs = softmax(np.array(outputs).reshape(samplesize, 2), axis=1)
            # the index of maximum in each line
            pred_outputs = np.argmax(px_outputs, axis = 1)
            real_outputs = np.array(x_outputs).reshape(samplesize,)

            acc = np.sum(pred_outputs == real_outputs)/samplesize

            genome.fitness = acc

    def run(config_file):
        # Load configuration.
        config = neat.Config(
            neat.DefaultGenome,
            neat.DefaultReproduction,
            neat.DefaultSpeciesSet,
            neat.DefaultStagnation,
            config_file,
        )

        # Create the population, which is the top-level object for a NEAT run.
        p = neat.Population(config)

        # add a stdout reporter to show progress in the terminal
        reporter = neat.StdOutReporter(False)
        p.add_reporter(reporter)
        stats = neat.StatisticsReporter()
        p.add_reporter(stats)
        #checkpointer = neat.Checkpointer(100)
        #p.add_reporter(checkpointer)
        # Run for up to 300 generations.
        winner = p.run(eval_genomes, generations)

        return [stats, winner]
    
    local_dir = os.getcwd()
    config_path = os.path.join(local_dir, "config-feedforward-letter2")
    [stats, winner] = run(config_path)
    
    return winner

In [6]:
#class1 = [0, 2, 4, 6, 8]
#winner1 = get_winner(class1)

In [7]:
winner_list = []
classes = [[0],[1],[2],[3],[4],[5],[6],[7],[8],[9],[10],[11],[12],[13],[14],[15],[16],[17],[18],[19],[20],[21],[22],[23],[24],[25]]

for i,class1 in enumerate(classes):
    print("running:{}/26".format(i))
    winner = get_winner(class1)
    winner_list.append(winner)

running:0/26
number of class1:64
number of class2:1536

 ****** Running generation 0 ****** 

Population's average fitness: 0.49443 stdev: 0.39642
Best fitness: 0.96125 - size: (2, 3) - species 5 - id 117
Average adjusted fitness: 0.454
Mean genetic distance 2.751, standard deviation 0.544
Population of 199 members in 15 species
Total extinctions: 0
Generation time: 2.974 sec

 ****** Running generation 1 ****** 

Population's average fitness: 0.69523 stdev: 0.36900
Best fitness: 0.96125 - size: (2, 4) - species 2 - id 227
Average adjusted fitness: 0.702
Mean genetic distance 2.803, standard deviation 0.559
Population of 200 members in 16 species
Total extinctions: 0
Generation time: 3.064 sec (3.019 average)

 ****** Running generation 2 ****** 

Population's average fitness: 0.77665 stdev: 0.32480
Best fitness: 0.96125 - size: (2, 3) - species 3 - id 117
Average adjusted fitness: 0.765
Mean genetic distance 2.841, standard deviation 0.553
Population of 202 members in 19 species
Total

Population's average fitness: 0.82681 stdev: 0.29668
Best fitness: 0.98687 - size: (4, 7) - species 3 - id 4026
Average adjusted fitness: 0.763
Mean genetic distance 3.003, standard deviation 0.600
Population of 203 members in 18 species
Total extinctions: 0
Generation time: 4.816 sec (4.253 average)

 ****** Running generation 25 ****** 

Population's average fitness: 0.82100 stdev: 0.29288
Best fitness: 0.99000 - size: (3, 6) - species 33 - id 4244
Average adjusted fitness: 0.786
Mean genetic distance 3.048, standard deviation 0.569
Population of 203 members in 19 species
Total extinctions: 0
Generation time: 5.147 sec (4.382 average)

 ****** Running generation 26 ****** 

Population's average fitness: 0.81002 stdev: 0.29507
Best fitness: 0.99000 - size: (3, 6) - species 33 - id 4244
Average adjusted fitness: 0.798
Mean genetic distance 2.992, standard deviation 0.563
Population of 202 members in 19 species
Total extinctions: 0
Generation time: 5.443 sec (4.545 average)

 ****** Run

Population's average fitness: 0.82252 stdev: 0.29860
Best fitness: 0.99125 - size: (13, 25) - species 42 - id 7230
Average adjusted fitness: 0.767
Mean genetic distance 2.995, standard deviation 0.535
Population of 200 members in 21 species
Total extinctions: 0
Generation time: 6.540 sec (6.522 average)

 ****** Running generation 49 ****** 

Population's average fitness: 0.81876 stdev: 0.29366
Best fitness: 0.99125 - size: (13, 25) - species 42 - id 7230
Average adjusted fitness: 0.764
Mean genetic distance 3.052, standard deviation 0.543
Population of 202 members in 22 species
Total extinctions: 0
Generation time: 5.989 sec (6.485 average)

 ****** Running generation 50 ****** 

Population's average fitness: 0.84329 stdev: 0.27358
Best fitness: 0.99125 - size: (13, 25) - species 42 - id 7230
Average adjusted fitness: 0.792
Mean genetic distance 3.077, standard deviation 0.535
Population of 200 members in 21 species
Total extinctions: 0
Generation time: 6.029 sec (6.454 average)

 ***

Population's average fitness: 0.81967 stdev: 0.26950
Best fitness: 0.99187 - size: (17, 36) - species 29 - id 8558
Average adjusted fitness: 0.773
Mean genetic distance 3.044, standard deviation 0.556
Population of 202 members in 15 species
Total extinctions: 0
Generation time: 7.195 sec (8.004 average)

 ****** Running generation 73 ****** 

Population's average fitness: 0.81712 stdev: 0.28264
Best fitness: 0.99187 - size: (17, 36) - species 29 - id 8558
Average adjusted fitness: 0.787
Mean genetic distance 3.012, standard deviation 0.569
Population of 203 members in 14 species
Total extinctions: 0
Generation time: 8.220 sec (8.168 average)

 ****** Running generation 74 ****** 

Population's average fitness: 0.85643 stdev: 0.24178
Best fitness: 0.99187 - size: (12, 24) - species 53 - id 12087
Average adjusted fitness: 0.833
Mean genetic distance 3.063, standard deviation 0.541
Population of 200 members in 16 species
Total extinctions: 0
Generation time: 8.918 sec (8.346 average)

 **

Population's average fitness: 0.84379 stdev: 0.27432
Best fitness: 0.99250 - size: (11, 24) - species 41 - id 12699
Average adjusted fitness: 0.789
Mean genetic distance 3.029, standard deviation 0.545
Population of 201 members in 19 species
Total extinctions: 0
Generation time: 7.590 sec (7.686 average)

 ****** Running generation 97 ****** 

Population's average fitness: 0.86535 stdev: 0.24748
Best fitness: 0.99250 - size: (11, 24) - species 41 - id 12699
Average adjusted fitness: 0.818
Mean genetic distance 3.079, standard deviation 0.551
Population of 195 members in 18 species
Total extinctions: 0
Generation time: 7.693 sec (7.697 average)

 ****** Running generation 98 ****** 

Population's average fitness: 0.82948 stdev: 0.27496
Best fitness: 0.99250 - size: (12, 27) - species 41 - id 15910
Average adjusted fitness: 0.792
Mean genetic distance 3.094, standard deviation 0.564
Population of 205 members in 18 species
Total extinctions: 0
Generation time: 7.883 sec (7.737 average)

 

...

Population's average fitness: 0.83386 stdev: 0.29322
Best fitness: 0.96937 - size: (5, 9) - species 4 - id 1112
Average adjusted fitness: 0.799
Mean genetic distance 2.823, standard deviation 0.597
Population of 199 members in 22 species
Total extinctions: 0
Generation time: 3.348 sec (3.156 average)

 ****** Running generation 11 ****** 

Population's average fitness: 0.85241 stdev: 0.26619
Best fitness: 0.96937 - size: (5, 9) - species 4 - id 1112
Average adjusted fitness: 0.820
Mean genetic distance 2.842, standard deviation 0.603
Population of 200 members in 22 species
Total extinctions: 0
Generation time: 3.330 sec (3.219 average)

 ****** Running generation 12 ****** 

Population's average fitness: 0.84577 stdev: 0.28434
Best fitness: 0.96937 - size: (5, 9) - species 4 - id 1112
Average adjusted fitness: 0.823
Mean genetic distance 2.837, standard deviation 0.603
Population of 200 members in 22 species
Total extinctions: 0
Generation time: 3.331 sec (3.268 average)

 ****** Runni

Population's average fitness: 0.81452 stdev: 0.31066
Best fitness: 0.97062 - size: (7, 10) - species 40 - id 4775
Average adjusted fitness: 0.724
Mean genetic distance 3.209, standard deviation 0.573
Population of 198 members in 22 species
Total extinctions: 0
Generation time: 4.768 sec (4.650 average)

 ****** Running generation 35 ****** 

Population's average fitness: 0.81913 stdev: 0.31509
Best fitness: 0.97062 - size: (7, 10) - species 40 - id 4775
Average adjusted fitness: 0.722
Mean genetic distance 3.202, standard deviation 0.568
Population of 206 members in 22 species
Total extinctions: 0
Generation time: 4.656 sec (4.678 average)

 ****** Running generation 36 ****** 

Population's average fitness: 0.77851 stdev: 0.35312
Best fitness: 0.97062 - size: (7, 10) - species 40 - id 4775
Average adjusted fitness: 0.685
Mean genetic distance 3.176, standard deviation 0.560
Population of 205 members in 22 species
Total extinctions: 0
Generation time: 4.849 sec (4.720 average)

 ******

Population's average fitness: 0.84973 stdev: 0.29319
Best fitness: 0.97062 - size: (6, 8) - species 50 - id 8888
Average adjusted fitness: 0.818
Mean genetic distance 3.018, standard deviation 0.548
Population of 201 members in 16 species
Total extinctions: 0
Generation time: 4.829 sec (5.063 average)

 ****** Running generation 59 ****** 

Population's average fitness: 0.84137 stdev: 0.28653
Best fitness: 0.97062 - size: (6, 8) - species 50 - id 8888
Average adjusted fitness: 0.818
Mean genetic distance 3.047, standard deviation 0.560
Population of 196 members in 15 species
Total extinctions: 0
Generation time: 4.970 sec (5.052 average)

 ****** Running generation 60 ****** 

Population's average fitness: 0.86523 stdev: 0.26329
Best fitness: 0.97062 - size: (6, 8) - species 50 - id 8888
Average adjusted fitness: 0.845
Mean genetic distance 3.042, standard deviation 0.554
Population of 203 members in 15 species
Total extinctions: 0
Generation time: 5.279 sec (5.064 average)

 ****** Ru

Population of 202 members in 17 species
Total extinctions: 0
Generation time: 5.140 sec (4.956 average)

 ****** Running generation 83 ****** 

Population's average fitness: 0.82122 stdev: 0.32322
Best fitness: 0.97062 - size: (13, 23) - species 71 - id 13298
Average adjusted fitness: 0.712
Mean genetic distance 3.058, standard deviation 0.575
Population of 200 members in 15 species
Total extinctions: 0
Generation time: 5.420 sec (4.983 average)

 ****** Running generation 84 ****** 

Population's average fitness: 0.81895 stdev: 0.32518
Best fitness: 0.97062 - size: (13, 23) - species 71 - id 13298
Average adjusted fitness: 0.742
Mean genetic distance 3.061, standard deviation 0.557
Population of 200 members in 16 species
Total extinctions: 0
Generation time: 5.457 sec (5.027 average)

 ****** Running generation 85 ****** 

Population's average fitness: 0.86275 stdev: 0.28052
Best fitness: 0.97062 - size: (13, 23) - species 71 - id 13298
Average adjusted fitness: 0.788
Mean genetic dis

Mean genetic distance 2.881, standard deviation 0.549
Population of 198 members in 18 species
Total extinctions: 0
Generation time: 5.260 sec (5.532 average)

 ****** Running generation 107 ****** 

Population's average fitness: 0.86006 stdev: 0.27488
Best fitness: 0.97125 - size: (21, 34) - species 88 - id 18373
Average adjusted fitness: 0.825
Mean genetic distance 2.885, standard deviation 0.534
Population of 200 members in 18 species
Total extinctions: 0
Generation time: 5.298 sec (5.562 average)

 ****** Running generation 108 ****** 

Population's average fitness: 0.90005 stdev: 0.23048
Best fitness: 0.97125 - size: (21, 34) - species 88 - id 18373
Average adjusted fitness: 0.862
Mean genetic distance 2.891, standard deviation 0.541
Population of 201 members in 19 species
Total extinctions: 0
Generation time: 5.124 sec (5.594 average)

 ****** Running generation 109 ****** 

Population's average fitness: 0.84749 stdev: 0.29476
Best fitness: 0.97125 - size: (21, 34) - species 88 - 

In [10]:
local_dir = os.getcwd()
config_path = os.path.join(local_dir, "config-feedforward-letter2")

config = neat.Config(
    neat.DefaultGenome,
    neat.DefaultReproduction,
    neat.DefaultSpeciesSet,
    neat.DefaultStagnation,
    config_path,
)

def get_pred_real(i, j):
    winner_net = neat.nn.FeedForwardNetwork.create(winner_list[i], config)
    
    numbers = [*range(26)]
    class1 = classes[i]
    class2 = []
    class1_indexes = []
    class2_indexes = []
    
    test_X = testX[j]
    test_y = testy[j]

    list_y = [test_y]
    digits_indexes = []
    for digit in numbers:
        li = [i for i in range(len(list_y)) if list_y[i] == digit]
        digits_indexes.extend([li])

    for i in numbers:
        if i in class1:
            class1_indexes.extend(digits_indexes[i])
        else:
            class2_indexes.extend(digits_indexes[i])
            class2.append(i)

    class1_label = [1] * len(class1_indexes)
    class2_label = [0] * len(class2_indexes)


    testsamplesize = 1
    test_x_inputs =[tuple(test_X)]
    test_x_outputs = [tuple([c]) for c in class1_label + class2_label]

    outputs = []
    for xi in test_x_inputs:
        output = winner_net.activate(xi)
        outputs.append(output)

    px_outputs = softmax(np.array(outputs).reshape(testsamplesize, 2), axis=1)
    # the index of maximum in each line
    pred_outputs = np.argmax(px_outputs, axis = 1)
    real_outputs = np.array(test_x_outputs).reshape(testsamplesize,)

    
    return [pred_outputs, real_outputs]

from random import randint
pred_value = []
error_list = []

for j in range(400):
    pred = []
    for i in range(26):
        [pred_outputs, real_outputs] = get_pred_real(i, j)
        #print(pred_outputs, real_outputs)
        pred.append(pred_outputs)
    #print(np.array(pred).T)
    pred_array = np.array(pred).reshape(26,)
    #print(pred_array)
    if (np.sum(pred_array) >= 1):
        pred_value.append(np.where(pred_array==1))
    else:
        pred_value.append(np.array(range(26)))

import random

list_P = []
for i in range(400):
    if (len(pred_value[i])== 1):
        #print(pred_value[i][0], digits.target[1617+i])
        list_P.append(random.sample(pred_value[i][0].tolist(),1)[0])
    else:
        list_P.append(random.sample(pred_value[i].tolist(),1)[0])
        
np.sum(list_P == testy)/400

0.2525

In [11]:
def get_pred_train(i, j):
    winner_net = neat.nn.FeedForwardNetwork.create(winner_list[i], config)
    
    numbers = [*range(26)]
    class1 = classes[i]
    class2 = []
    class1_indexes = []
    class2_indexes = []
    
    test_X = X[j]
    test_y = y[j]

    list_y = [test_y]
    digits_indexes = []
    for digit in numbers:
        li = [i for i in range(len(list_y)) if list_y[i] == digit]
        digits_indexes.extend([li])

    for i in numbers:
        if i in class1:
            class1_indexes.extend(digits_indexes[i])
        else:
            class2_indexes.extend(digits_indexes[i])
            class2.append(i)

    class1_label = [1] * len(class1_indexes)
    class2_label = [0] * len(class2_indexes)


    testsamplesize = 1
    test_x_inputs =[tuple(test_X)]
    test_x_outputs = [tuple([c]) for c in class1_label + class2_label]

    outputs = []
    for xi in test_x_inputs:
        output = winner_net.activate(xi)
        outputs.append(output)

    px_outputs = softmax(np.array(outputs).reshape(testsamplesize, 2), axis=1)
    # the index of maximum in each line
    pred_outputs = np.argmax(px_outputs, axis = 1)
    real_outputs = np.array(test_x_outputs).reshape(testsamplesize,)

    
    return [pred_outputs, real_outputs]

pred_value = []
error_list = []

for j in range(1600):
    pred = []
    for i in range(26):
        [pred_outputs, real_outputs] = get_pred_train(i, j)
        #print(pred_outputs, real_outputs)
        pred.append(pred_outputs)
    #print(np.array(pred).T)
    pred_array = np.array(pred).reshape(26,)
    #print(pred_array)
    if (np.sum(pred_array) >= 1):
        pred_value.append(np.where(pred_array==1))
    else:
        pred_value.append(np.array(range(26)))

import random

list_P = []
for i in range(1600):
    if (len(pred_value[i])== 1):
        #print(pred_value[i][0], digits.target[1617+i])
        list_P.append(random.sample(pred_value[i][0].tolist(),1)[0])
    else:
        list_P.append(random.sample(pred_value[i].tolist(),1)[0])
print("Trainset acc:{}".format(np.sum(list_P == y)/1600))
winner_fitness = []
for winner in winner_list:
    #print(winner.fitness)
    winner_fitness.append(winner.fitness)
print("Avg Base Learner:{}".format(np.mean(winner_fitness)))

Trainset acc:0.306875
Avg Base Learner:0.9709375


In [12]:
# program to check if there is exist a path between two vertices 
# of a graph 

from collections import defaultdict 

#This class represents a directed graph using adjacency list representation 
class Graph: 

	def __init__(self,vertices): 
		self.V= vertices #No. of vertices 
		self.graph = defaultdict(list) # default dictionary to store graph 

	# function to add an edge to graph 
	def addEdge(self,u,v): 
		self.graph[u].append(v) 
	
	# Use BFS to check path between s and d 
	def isReachable(self, s, d): 
		# Mark all the vertices as not visited 
		visited =[False]*(self.V) 

		# Create a queue for BFS 
		queue=[] 

		# Mark the source node as visited and enqueue it 
		queue.append(s) 
		visited[s] = True

		while queue: 

			#Dequeue a vertex from queue 
			n = queue.pop(0) 
			
			# If this adjacent node is the destination node, 
			# then return true 
			if n == d: 
				return True

			# Else, continue to do BFS 
			for i in self.graph[n]: 
				if visited[i] == False: 
					queue.append(i) 
					visited[i] = True
		# If BFS is complete without visited d 
		return False
    
    
def findAllPath(graph,start,end,path=[]):
    path = path +[start]
    if start == end:
        return [path]
 
    paths = [] #存储所有路径    
    for node in graph[start]:
        if node not in path:
            newpaths = findAllPath(graph,node,end,path) 
            for newpath in newpaths:
                paths.append(newpath)
    return paths

numbers = [*range(2)]
number_of_classification = len(numbers)

list_nodes_number = []
list_connection_number = []

for winner in winner_list:
    used_nodes = list(winner.nodes.keys())
    # create a dict for mapping 
    l1 = numbers + config.genome_config.input_keys + used_nodes[number_of_classification:]
    l2 = range(len(l1))
    dict_nodes = dict(zip(l1, l2))

    ### 将节点加入图
    g = Graph(len(l1))# inputs + outputs + used  
    # add connections in the graph
    for cg in winner.connections.values():
        if cg.enabled:
            g.addEdge(dict_nodes[cg.key[0]],dict_nodes[cg.key[1]])

    v = numbers
    list_connections = []
    for vi in v:
        for u in range(number_of_classification, len(config.genome_config.input_keys) + number_of_classification):
            list_connections.append(g.isReachable(u, vi))

    nodes_inputs = range(number_of_classification, number_of_classification+ 64)
    nodes_outputs = range(number_of_classification)
    nodes_mid = range(number_of_classification + 64, len(dict_nodes))

    # 找到所有输入和输出连接的路
    all_path = []
    for u in nodes_inputs:
        for v in nodes_outputs:
            path = findAllPath(g.graph, u, v)
            if path:
                all_path = all_path + path

    # 得到最长路，以及每个节点在哪个层级
    max_length = max([len(x) for x in all_path])
    nodes_tuples_list = []
    for path in all_path:
        #print(path)
        for node in path:
            nodes_tuples_list.append([node, path.index(node)])

    # 确定节点的层级（消除重复）       
    nodes_tuples_fixed_list = []
    for index in range(max_length):
        for nodes in nodes_tuples_list:
            node, node_index = nodes[0], nodes[1]
            if node_index == index: 
                if nodes not in nodes_tuples_fixed_list:
                    nodes_tuples_fixed_list.append(nodes)


    # 所有有连接的节点
    all_used_nodes = []
    for node in nodes_tuples_fixed_list:
        if node[0] not in all_used_nodes:
            all_used_nodes.append(node[0])

    # 确定节点的层级（最终确定）
    nodes_tuples_fixed_list_final = []

    for node in all_used_nodes:
        if node in nodes_inputs:
            nodes_tuples_fixed_list_final.append([node, 0])
        elif node in nodes_outputs:
            nodes_tuples_fixed_list_final.append([node, max_length - 1])
        else:
            list_node = []
            for nodes in nodes_tuples_fixed_list:
                if node == nodes[0]:
                    list_node.append(nodes)
            layer = 0
            for ele in list_node:
                # 删除重复项，取最大
                if ele[1] > layer:
                    layer = ele[1]
            nodes_tuples_fixed_list_final.append([node,layer])


    # 每个层级的节点合数
    length_of_layers = []
    number_of_nodes = 0
    for index in range(max_length):
        for nodes in nodes_tuples_fixed_list_final:
            node, node_index = nodes[0], nodes[1]
            if node_index == index: 
                number_of_nodes += 1
        length_of_layers.append(number_of_nodes)

    length_of_each_layer = []
    length_of_each_layer.append(length_of_layers[0])
    for i in range(1, max_length):
        length_of_each_layer.append(length_of_layers[i] - length_of_layers[i-1])

    # 输出每个层级的节点个数
    #print("length of each layers:", length_of_each_layer)

    # 所有端到端的路
    all_path_side2side = []
    for path in all_path:
        if len(path) == 2:
            all_path_side2side.append(path)
        else:
            for i in range(len(path)-1):
                all_path_side2side.append([path[i],path[i+1]])

    # 定义节点到绘图的字典
    dict_nodes_graph = dict()
    count_number_layer = [0] * max_length
    for nodes in nodes_tuples_fixed_list_final:
        node, layer = nodes
        dict_nodes_graph[node] = [count_number_layer[layer], layer]
        count_number_layer[layer] += 1
    list_nodes_number.append(np.sum(length_of_each_layer))
    list_connection_number.append(len(all_path_side2side))
    #print("Number of nodes:{} Number of connections:{}".format(np.sum(length_of_each_layer),len(all_path_side2side)))
print("Total nodes:{} Total connections::{}".format(np.sum(list_nodes_number), np.sum(list_connection_number)))

Total nodes:303 Total connections::861
